In [3]:

import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.layers import merge
# from keras.layers.merge import Concatenate
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.normalization import BatchNormalization

from keras.optimizers import Adam, SGD
import keras
print keras.__version__


1.1.1


In [4]:
import numpy as np
import h5py
import scipy.io
validmat = scipy.io.loadmat('/Users/jx/Documents/FACSseq/YFSIVc/sTRSV_valid.mat')
testmat = scipy.io.loadmat('/Users/jx/Documents/FACSseq/YFSIVc/sTRSV_valid.mat')
trainmat = scipy.io.loadmat('/Users/jx/Documents/FACSseq/YFSIVc/sTRSV_train.mat')


X_train = np.transpose(np.array(trainmat['tr'][0][0][0]),axes=(0,2,1))
# X_train = np.expand_dims(X_train,axis=(2))
y_train = np.array(trainmat['tr'][0][0][1]).squeeze()
# y_train = y_train.reshape((-1, 1))
# y_train.squeeze()
print X_train.shape
print y_train.shape



(2385, 4, 113)
(2385,)


In [6]:


lr = 1e-5#learning rate
reg = 1e-3
print 'building model'
nb_filters=32
model = Sequential()
model.add(LSTM(256,  W_regularizer=l2(reg),return_sequences=True, input_shape=(4,113)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(LSTM(128,  W_regularizer=l2(reg),return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(LSTM(128,  W_regularizer=l2(reg),return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(LSTM(64,  W_regularizer=l2(reg),return_sequences=True))
model.add(Activation('relu'))
model.add(Dropout(0.2))



# model.add(Convolution1D(8,64, border_mode='same', W_regularizer=l2(reg),input_shape=(4,113))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.1))

# model.add(Convolution1D(4,32, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.1))

# model.add(Convolution1D(4,32, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.1))
# model.add(Convolution1D(4,32, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.1))
# model.add(Convolution1D(8,64, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.1))

# model.add(Convolution1D(4,32, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
# model.add(Activation('relu'))
# model.add(Dropout(0.05))


model.add(Flatten())


model.add(Dense(1000))
model.add(Dense(1))
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

print 'compiling model'
model.compile(loss='mse', optimizer='adam', metrics=["mse"])


checkpointer = ModelCheckpoint(filepath="bestmodel3D.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

X_valid=np.transpose(validmat['tr'][0][0][0],axes=(0,2,1))
# X_valid=np.expand_dims(X_valid)
y_valid=np.array(validmat['tr'][0][0][1]).squeeze()


history=model.fit(X_train, y_train, batch_size=512, nb_epoch=500, shuffle=True, validation_data=(X_valid, y_valid),callbacks=[checkpointer,earlystopper])



building model
compiling model
Train on 2385 samples, validate on 265 samples
Epoch 1/500
2385/2385 [==============================] - 1s - loss: 1.0075 - mean_squared_error: 1.0054 - val_loss: 0.9421 - val_mean_squared_error: 0.9421
Epoch 2/500
2385/2385 [==============================] - 1s - loss: 0.9935 - mean_squared_error: 0.9914 - val_loss: 0.9250 - val_mean_squared_error: 0.9250
Epoch 3/500
2385/2385 [==============================] - 1s - loss: 0.9361 - mean_squared_error: 0.9340 - val_loss: 0.8735 - val_mean_squared_error: 0.8735
Epoch 4/500
2385/2385 [==============================] - 1s - loss: 0.8334 - mean_squared_error: 0.8314 - val_loss: 0.9263 - val_mean_squared_error: 0.9263
Epoch 5/500
2385/2385 [==============================] - 1s - loss: 0.7889 - mean_squared_error: 0.7870 - val_loss: 0.8543 - val_mean_squared_error: 0.8543
Epoch 6/500
2385/2385 [==============================] - 1s - loss: 0.7507 - mean_squared_error: 0.7488 - val_loss: 0.7652 - val_mean_squared_

In [4]:
%matplotlib

out=model.predict(X_train, batch_size=512,verbose=1)
plt.plot(y_train,out,'ro')

model.summary()


Using matplotlib backend: MacOSX
31685/31685 [==============================] - 5s     
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 4, 256)        378880      lstm_input_1[0][0]               
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 4, 256)        0           lstm_1[0][0]                     
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 4, 256)        0           activation_1[0][0]               
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 4, 128)        197120      dropout_1[0][0]                  
___

In [5]:
import scipy
out=out.squeeze()
print out.shape
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(y_train, out)
print r_value
print slope
print intercept

(31685,)
0.845447373944
0.578251850801
-0.0150249163664


In [12]:
%matplotlib

X_test=np.transpose(testmat['tr'][0][0][0],axes=(0,2,1))
y_test=np.array(testmat['tr'][0][0][1]).squeeze()

y_out=model.predict(X_test, batch_size=512,verbose=1)
plt.plot(y_test,y_out,'ro')
plt.ylabel('model prediction')
plt.xlabel('FACS-seq data')

model.summary()

Using matplotlib backend: MacOSX
3521/3521 [==============================] - 0s     
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_5 (LSTM)                    (None, 4, 256)        378880      lstm_input_2[0][0]               
____________________________________________________________________________________________________
activation_5 (Activation)        (None, 4, 256)        0           lstm_5[0][0]                     
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 4, 256)        0           activation_5[0][0]               
____________________________________________________________________________________________________
lstm_6 (LSTM)                    (None, 4, 128)        197120      dropout_5[0][0]                  
_____

In [13]:
import scipy
out=y_out.squeeze()
print out.shape
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(y_test, out)
print r_value
print slope
print intercept


(3521,)
0.614309069504
0.446675731966
0.0750343296178


In [8]:
print history.history.keys()

['mean_squared_error', 'loss', 'val_mean_squared_error', 'val_loss']


In [9]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()